In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences
from transformers import XLNetForSequenceClassification, AdamW, XLNetConfig, XLNetTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import nltk.data
import progressbar
import itertools
from numpy import load

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = "/content/Drive/My Drive/XLNet_right_model/"
tokenizer = XLNetTokenizer.from_pretrained(model_dir)
model = XLNetForSequenceClassification.from_pretrained(model_dir, output_hidden_states=True)
model.to(device)

In [ ]:
nltk.download('punkt')
nltk_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df = pd.read_csv("/content/Drive/My Drive/LNLP/anno_dataset.csv")
pred_labels = [1,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0]
#model, tokenizer, pred labels and all the libraries are ready

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

chunk_scores = load("/content/Drive/My Drive/LNLP/EXP/Hierexp/XLNet_2layer_noatt_3epoch/occwts.npy", allow_pickle = True)
chunk_scores = list(chunk_scores)

In [ ]:
def xlnet_tokenize(sents, tokenizer):
  tok_sents = []
  for sen in sents:
    tok_sents.append(tokenizer.tokenize(sen))

  return tok_sents




def sentence_marker(tokenized_sents):
  marker_array = []
  sent_num=1
  for tokenized_sentence in tokenized_sents:
    sentence_marker = []
    for i in range(len(tokenized_sentence)):
      if(i==0):
        sentence_marker.append(sent_num)
      else:
        sentence_marker.append(0)

    sent_num+=1
    marker_array.append(sentence_marker)

  return marker_array




def chunked_tokens_maker(all_toks, markers):
  splitted_toks = []
  splitted_markers = []
  l=0
  r=510
  while(l<len(all_toks)):
    splitted_toks.append(all_toks[l:min(r,len(all_toks))])
    splitted_markers.append(markers[l:min(r,len(markers))])
    l+=410
    r+=410

  return splitted_toks, splitted_markers




def calculate_num_of_sents(chunk_marker_list):
  ct=0
  for i in range(len(chunk_marker_list)):
    if(chunk_marker_list[i] != 0):
      ct+=1
  
  return ct-1



def sentence_tokens_maker(marks, chunk_toks):
  pair_of_ids = []
  st = -1000
  ed = -1000
  for i,mark in enumerate(marks):
    if(mark==-777):
      st=i
    
    if(mark!=-777 and mark!=777 and mark!=0):
      ed=i-1
      pair_of_ids.append((st,ed))
      st=i

    if(mark==777):
      ed=i
      pair_of_ids.append((st,ed))

  return pair_of_ids




def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks



def get_output_for_one_vec(input_id, att_mask):
  input_ids = torch.tensor(input_id)
  att_masks = torch.tensor(att_mask)
  input_ids = input_ids.unsqueeze(0)
  att_masks = att_masks.unsqueeze(0)
  model.eval()
  input_ids = input_ids.to(device)
  att_masks = att_masks.to(device)
  with torch.no_grad():
      logits, encoded_layers = model(input_ids=input_ids, token_type_ids=None, attention_mask=att_masks)

  return logits




def get_XLNet_output_logits(encoded_sents, tokenizer, model):
  e_sents = []
  e_sents.append(encoded_sents)
  e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="pre")
  att_masks = att_masking(e_sents)
  return get_output_for_one_vec(e_sents[0], att_masks[0])


def xlnet_detok(xtoks):
  xtoks[0] = "_" + xtoks[0]
  words = []
  word = ""
  for tok in xtoks:
    if(tok[0] == "▁"):
      words.append(word)
      word = tok[1:]
    else:
      word+=tok

  output = ""
  for word in words[1:]:
    output += (word + " ")

  return output

In [ ]:
def get_explanation(chunked_xlnet_tokens, chunked_markers, chunk_scores, doc_num, tokenizer, predicted_label):
    explanation = ""
    for chunk_number,score in enumerate(chunk_scores[doc_num]):
        if(chunk_number == 0):
          chunked_markers[chunk_number][0] = -777
          chunked_markers[chunk_number][-1] = 777
        else:
          if(len(chunked_markers[chunk_number]) < 101):
            continue
          chunked_markers[chunk_number][100] = -777
          chunked_markers[chunk_number][-1] = 777

        if(score < 0):
          continue

        ct_sent = calculate_num_of_sents(chunked_markers[chunk_number])
        #print("ct_sent: {:}".format(ct_sent))
        top_k = 0.4*ct_sent
        dict_sent_to_score = {}
        #print(len(chunked_markers[chunk_number]))
        pair_of_ids = sentence_tokens_maker(chunked_markers[chunk_number], chunked_xlnet_tokens[chunk_number])
        #print(pair_of_ids)
        # lens = [len(t) for t in tokens_as_sentences_list]
        # print(str(sum(lens)) + "\n")

        CLS = tokenizer.cls_token
        SEP = tokenizer.sep_token
        PAD = tokenizer.pad_token

        original_logits = get_XLNet_output_logits(tokenizer.convert_tokens_to_ids(chunked_xlnet_tokens[chunk_number] + [SEP] + [CLS]), tokenizer, model)
        original_score = float(original_logits[0][predicted_label])

        for i in range(len(pair_of_ids)):
            if(pair_of_ids[i][0] == -1000):
              pair_of_ids[i] = (0,pair_of_ids[i][1]) 
            normalizing_length = pair_of_ids[i][1] - pair_of_ids[i][0] + 1
            if(normalizing_length == 0):
              continue
            pad_sentence = [PAD]*normalizing_length
            #print("pad_length = {:}".format(len(pad_sentence)))
            left = []
            right = []
            left = chunked_xlnet_tokens[chunk_number][:pair_of_ids[i][0]]
            right = chunked_xlnet_tokens[chunk_number][pair_of_ids[i][1]+1:]
        
            final_tok_sequence = left + pad_sentence + right + [SEP] + [CLS]
            #print("final_tok_sequence = {:}".format(len(final_tok_sequence)))
            encoded_sents = tokenizer.convert_tokens_to_ids(final_tok_sequence)
            logits = get_XLNet_output_logits(encoded_sents, tokenizer, model)
            score_for_predicted_label = float(logits[0][predicted_label])

            sent_score = 100

            if(score_for_predicted_label > original_score):
              sent_score = -1*(score_for_predicted_label - original_score)
            else:
              sent_score = original_score - score_for_predicted_label

            sent_score_norm = sent_score/normalizing_length
            sentence_in_words = xlnet_detok(chunked_xlnet_tokens[chunk_number][pair_of_ids[i][0]:pair_of_ids[i][1]+1])
            dict_sent_to_score[sentence_in_words] = sent_score_norm

        sort_scores = sorted(dict_sent_to_score.items(), key=lambda x: x[1], reverse=True)
        sorted_sentences =[]
        for i in sort_scores:
          sorted_sentences.append(i[0])

        for sentence in sorted_sentences[:int(top_k)]:
          explanation+=sentence

      
    return explanation

In [ ]:
final_explanations = {}
for i in progressbar.progressbar(range(len(df["text"]))):
  doc_text = df["text"].iloc[i]
  doc_name = df["name"].iloc[i]
  sents = nltk_tokenizer.tokenize(doc_text)
  xlnet_tokenized_sents = xlnet_tokenize(sents, tokenizer)
  marked_tokenized_sents = sentence_marker(xlnet_tokenized_sents)
  xlnet_tokens = list(itertools.chain.from_iterable(xlnet_tokenized_sents))
  markers = list(itertools.chain.from_iterable(marked_tokenized_sents))
  if(len(xlnet_tokens) > 10000):
    xlnet_tokens = xlnet_tokens[len(xlnet_tokens)-10000:]
    markers = markers[len(markers)-10000:]

  chunked_xlnet_tokens, chunked_markers = chunked_tokens_maker(xlnet_tokens, markers)

  explanation_of_this_doc = get_explanation(chunked_xlnet_tokens, chunked_markers, chunk_scores, i, tokenizer, pred_labels[i])
  final_explanations[doc_name] = explanation_of_this_doc

100% (56 of 56) |########################| Elapsed Time: 0:10:38 Time:  0:10:38


In [ ]:
import json
json_file = open("occ_exp_full.json", "w")
json.dump(final_explanations, json_file)

In [ ]:
json_file = open("occ_exp_full.json","r")
data = json.load(json_file)

In [ ]:
print(data)

{'1962_213.txt': 'order to substantiate the charge against him the complainant number respondents examined himself and a number of also adduced in evidence a certain document marketed of the companystitution is directed against the order of a division bench of the calcutta high companyrt dated december 21 1960 setting aside the order of acquittal passed by the trial magistrate dated july 2 1958. we heard this appeal on the eve of the long vacation and pronumbernced our order to the effect that the appeal was allowed and the order of acquittal was to stand and that reasons would be given appellant defense was that the case against him was false and that he had been falsely implicated for reasons which need number be to him as a lawyer on behalf of the order to substantiate the charge against him the complainant number respondents examined himself and a number of also adduced in evidence a certain document marketed it companyld have pronumbernced its judgment one way or the other on the 